<a href="https://colab.research.google.com/github/tanyagupta1/Machine-Learning/blob/main/BTP/comment_hierarchy_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import stuff
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
%matplotlib inline
#https://python.plainenglish.io/how-to-use-pandas-profiling-on-google-colab-e34f34ff1c9f

In [ ]:
path = "/content/drive/MyDrive/IIIT_DATA/BTP Data/clean_posts/"

In [ ]:
"""
# Uncomment this to create copies in a folder

subs = ["ADHD","Anger","Anxiety","BPD","OCD","OpiatesRecovery","SuicideWatch","addiction","alcoholicsanonymous","depression","domesticviolence","eating_disorders","getting_over_it","helpmecope","mentalillness","rapecounseling","sad","selfharm","selfhelp","socialanxiety"]
import shutil
for topic in subs:
    src= "/content/drive/MyDrive/IIIT_DATA/BTP Data/"+topic+"_com.csv"
    dst= "/content/drive/MyDrive/IIIT_DATA/BTP Data/clean_posts/"
    shutil.copy(src, dst)
"""

'\n# Uncomment this to create copies in a folder\n\nsubs = ["ADHD","Anger","Anxiety","BPD","OCD","OpiatesRecovery","SuicideWatch","addiction","alcoholicsanonymous","depression","domesticviolence","eating_disorders","getting_over_it","helpmecope","mentalillness","rapecounseling","sad","selfharm","selfhelp","socialanxiety"]\nimport shutil\nfor topic in subs:\n    src= "/content/drive/MyDrive/IIIT_DATA/BTP Data/"+topic+"_com.csv"\n    dst= "/content/drive/MyDrive/IIIT_DATA/BTP Data/clean_posts/"\n    shutil.copy(src, dst)\n'

In [ ]:
subs = ["ADHD","Anger","Anxiety","BPD","OCD","OpiatesRecovery","SuicideWatch","addiction","alcoholicsanonymous","depression","domesticviolence","eating_disorders","getting_over_it","helpmecope","mentalillness","rapecounseling","sad","selfharm","selfhelp","socialanxiety"]

post_df, comment_df = pd.DataFrame(), pd.DataFrame()

for topic in subs:
    topic = path+topic
    tmp_post_df = pd.read_csv(topic+"_clean.csv")
    # tmp_post_df = pd.read_csv(topic+"_sub.csv")
    tmp_com_df = pd.read_csv(topic+"_com.csv")
    post_df=post_df.append(tmp_post_df,ignore_index=True)
    comment_df=comment_df.append(tmp_com_df,ignore_index=True)

## An example of posts and comment

In [ ]:
post_df.head()

,Unnamed: 0,sub_id,title,body,author,score,awards,numComms,created,subreddit
0,0,ei6uyt,Ignoring/oblivious of your own mental state?,I've just recently been diagnosed (ADHD-PI) in...,g_b_0_t,1,0,3,2019-12-31 18:37:49,ADHD
1,1,ei6wbx,Excrutiating urgency to get out of the car.,Does anyone else feel an excrutiating urgency ...,amberaubade,1,0,7,2019-12-31 18:40:32,ADHD
2,2,ei6xa3,Happy new year my guys love you all x,That's the decade done! Bloody hell it's been ...,rstar345,1,0,0,2019-12-31 18:42:26,ADHD
3,3,ei7dle,How much has the mandatory drug tests usually ...,I finally decided to get back on meds (haven’t...,Bgeaz,1,0,5,2019-12-31 19:14:50,ADHD
4,4,ei7ee5,Recently Diagnosed- any advice?,I am currently in my 2nd year of undergrad and...,alyssaj99,1,0,2,2019-12-31 19:16:27,ADHD


In [ ]:
comment_df.head()

,post_id,body,author,is_op,created,score,awards,comment_id,parent_id
0,ei6uyt,I think that's actually a pretty common type o...,crisis_loitering,False,2020-01-01 05:26:15,1,0,fcpaoys,t3_ei6uyt
1,ei6uyt,Thanks for this - I have been thinking about j...,g_b_0_t,True,2019-12-31 21:33:25,1,0,fcob8hk,t1_fco58cc
2,ei6uyt,Try journaling or writing a diary. I like usi...,heywire84,False,2019-12-31 20:31:55,1,0,fco58cc,t3_ei6uyt
3,ei6wbx,I'm glad you know you're not alone!,amberaubade,True,2020-01-10 00:13:17,1,0,fdoxlrn,t1_fdl6xv3
4,ei6wbx,I really thought I was alone in this and that ...,BogusDou,False,2020-01-08 21:14:44,1,0,fdl6xv3,t3_ei6wbx


In [ ]:
#
#
# refer to this for this post id only: https://www.reddit.com/r/ADHD/comments/ei6uyt/
# for better understanding
#
comment_df[comment_df['post_id']=='ei6uyt']

,post_id,body,author,is_op,created,score,awards,comment_id,parent_id
0,ei6uyt,I think that's actually a pretty common type o...,crisis_loitering,False,2020-01-01 05:26:15,1,0,fcpaoys,t3_ei6uyt
1,ei6uyt,Thanks for this - I have been thinking about j...,g_b_0_t,True,2019-12-31 21:33:25,1,0,fcob8hk,t1_fco58cc
2,ei6uyt,Try journaling or writing a diary. I like usi...,heywire84,False,2019-12-31 20:31:55,1,0,fco58cc,t3_ei6uyt


## Continued

In [ ]:
post_df = post_df.drop(post_df[(post_df.body=="[removed]")].index)
post_df = post_df.drop(post_df[(post_df.body=="[deleted]")].index)
post_df = post_df.drop(post_df[(post_df.author=="[deleted]")].index)

comment_df = comment_df.drop(comment_df[(comment_df.body=="[removed]")].index)
comment_df = comment_df.drop(comment_df[(comment_df.body=="[deleted]")].index)
comment_df = comment_df.drop(comment_df[(comment_df.author=="[deleted]")].index)
# post_df=post_df.drop(columns='Unnamed: 0')
post_df['body'].fillna("",inplace=True)
comment_df['body'].fillna("",inplace=True)

In [ ]:
comment_df=comment_df.reset_index(drop=True)

In [ ]:
print(len(post_df),len(comment_df))

9624 55293


In [ ]:
all_post_ids=set(post_df['sub_id'])
len(all_post_ids)

9624

In [ ]:
post_df.head()

,Unnamed: 0,sub_id,title,body,author,score,awards,numComms,created,subreddit
0,0,ei6uyt,Ignoring/oblivious of your own mental state?,I've just recently been diagnosed (ADHD-PI) in...,g_b_0_t,1,0,3,2019-12-31 18:37:49,ADHD
1,1,ei6wbx,Excrutiating urgency to get out of the car.,Does anyone else feel an excrutiating urgency ...,amberaubade,1,0,7,2019-12-31 18:40:32,ADHD
2,2,ei6xa3,Happy new year my guys love you all x,That's the decade done! Bloody hell it's been ...,rstar345,1,0,0,2019-12-31 18:42:26,ADHD
3,3,ei7dle,How much has the mandatory drug tests usually ...,I finally decided to get back on meds (haven’t...,Bgeaz,1,0,5,2019-12-31 19:14:50,ADHD
4,4,ei7ee5,Recently Diagnosed- any advice?,I am currently in my 2nd year of undergrad and...,alyssaj99,1,0,2,2019-12-31 19:16:27,ADHD


In [ ]:
comment_df.head()

,post_id,body,author,is_op,created,score,awards,comment_id,parent_id
0,ei6uyt,I think that's actually a pretty common type o...,crisis_loitering,False,2020-01-01 05:26:15,1,0,fcpaoys,t3_ei6uyt
1,ei6uyt,Thanks for this - I have been thinking about j...,g_b_0_t,True,2019-12-31 21:33:25,1,0,fcob8hk,t1_fco58cc
2,ei6uyt,Try journaling or writing a diary. I like usi...,heywire84,False,2019-12-31 20:31:55,1,0,fco58cc,t3_ei6uyt
3,ei6wbx,I'm glad you know you're not alone!,amberaubade,True,2020-01-10 00:13:17,1,0,fdoxlrn,t1_fdl6xv3
4,ei6wbx,I really thought I was alone in this and that ...,BogusDou,False,2020-01-08 21:14:44,1,0,fdl6xv3,t3_ei6wbx


In [ ]:
from tqdm import tqdm
final_num_com={}
no_of_unique_posters={}

for id in tqdm(all_post_ids):
  spec_com = comment_df[comment_df['post_id']==id]
  unique_posters = len(set(spec_com['author']))
  final_num_com[id]=len(spec_com)
  no_of_unique_posters[id]=unique_posters

post_df['actual_com'] = post_df['sub_id'].map(final_num_com)
post_df['no_of_posters'] = post_df['sub_id'].map(no_of_unique_posters)

100%|██████████| 9624/9624 [00:34<00:00, 281.57it/s]


In [ ]:
op_comments = comment_df[comment_df['is_op']==True]
###################
interactive_post_ids = set(op_comments['post_id']).intersection(all_post_ids)

In [ ]:
len(interactive_post_ids)

4161

In [ ]:
isolated_post_submissions = post_df[post_df['actual_com']==0]
### -------------
isolated_post_ids = set(isolated_post_submissions['sub_id']).intersection(all_post_ids)
len(isolated_post_ids)

1857

In [ ]:
####################
non_interactive_ids=all_post_ids-isolated_post_ids-interactive_post_ids
len(non_interactive_ids)

3606

In [ ]:
"""
interact & non interact
sim (post, all root comments) -> in case 1 post => 4 root comments =>> [s1 s2 s3 s4] -> average [s_avg]

HYPO: cosine-similiarity

-> divide among int, non int, and iso

"""

'\ninteract & non interact\nsim (post, all root comments) -> in case 1 post => 4 root comments =>> [s1 s2 s3 s4] -> average [s_avg]\n\nHYPO: cosine-similiarity\n\n-> divide among int, non int, and iso\n\n'

In [ ]:
post_df.head(3)

,Unnamed: 0,sub_id,title,body,author,score,awards,numComms,created,subreddit,actual_com,no_of_posters
0,0,ei6uyt,Ignoring/oblivious of your own mental state?,I've just recently been diagnosed (ADHD-PI) in...,g_b_0_t,1,0,3,2019-12-31 18:37:49,ADHD,3,3
1,1,ei6wbx,Excrutiating urgency to get out of the car.,Does anyone else feel an excrutiating urgency ...,amberaubade,1,0,7,2019-12-31 18:40:32,ADHD,6,6
2,2,ei6xa3,Happy new year my guys love you all x,That's the decade done! Bloody hell it's been ...,rstar345,1,0,0,2019-12-31 18:42:26,ADHD,0,0


In [ ]:
comment_df.head(3)

,post_id,body,author,is_op,created,score,awards,comment_id,parent_id
0,ei6uyt,I think that's actually a pretty common type o...,crisis_loitering,False,2020-01-01 05:26:15,1,0,fcpaoys,t3_ei6uyt
1,ei6uyt,Thanks for this - I have been thinking about j...,g_b_0_t,True,2019-12-31 21:33:25,1,0,fcob8hk,t1_fco58cc
2,ei6uyt,Try journaling or writing a diary. I like usi...,heywire84,False,2019-12-31 20:31:55,1,0,fco58cc,t3_ei6uyt


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from statistics import mean
import torch
!pip install transformers sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

     |████████████████████████████████| 3.8 MB 28.2 MB/s 
     |████████████████████████████████| 79 kB 11.5 MB/s 
     |████████████████████████████████| 596 kB 61.7 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 6.5 MB 63.6 MB/s 
     |████████████████████████████████| 895 kB 63.6 MB/s 
     |████████████████████████████████| 1.2 MB 66.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=b948f8f6c98e53869c8a7b971213043eff163b7a5a3ec1bb931657157f09593c
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
from tqdm import tqdm
sims = {}
def cos_sim(post, coms):
    if len(coms)<1: return 0
    s = cosine_similarity([post], coms[:])
    return mean(s[0])

for index, row in tqdm(post_df.iterrows()):
    if row['sub_id'] in isolated_post_ids: continue
    if row['sub_id'] in sims.keys(): continue
    else:
        coms = comment_df[comment_df['post_id']==row['sub_id']]
        a = row['title']+" "+row['body']
        a = a.lower()
        a = model.encode(a)
        
        b = list(coms['body'])
        for i,each in enumerate(list(b)):
            b[i] = each.lower()
        b = model.encode(b)

        sims[row['sub_id']] = cos_sim(a,b)

9624it [08:43, 18.39it/s]


In [ ]:
import pickle
  
try:
    geeky_file = open('sims', 'wb')
    pickle.dump(sims, geeky_file)
    geeky_file.close()
  
except:
    print("Something went wrong")

In [ ]:
i,int_sim_sum = 0,0
j,non_int_sim_sum = 0,0
for id in sims.keys():
    if id in interactive_post_ids:
        int_sim_sum+=sims[id]
        i+=1
    elif id in non_interactive_ids:
        non_int_sim_sum+=sims[id]
        j+=1

int_avgsim = int_sim_sum/i
nonint_avgsim = non_int_sim_sum/j

In [ ]:
int_avgsim, nonint_avgsim

(0.5138316015106136, 0.5340126354000166)

In [ ]:
def category(sub_id):
  if(sub_id in interactive_post_ids):
    return 'interactive'
  elif(sub_id in isolated_post_ids):
    return 'isolated'
  else:
    return 'non-interactive'

In [ ]:
post_df['post_category'] = post_df['sub_id'].apply(category)

In [ ]:
post_df.head()

,Unnamed: 0,sub_id,title,body,author,score,awards,numComms,created,subreddit,actual_com,no_of_posters,post_category
0,0,ei6uyt,Ignoring/oblivious of your own mental state?,I've just recently been diagnosed (ADHD-PI) in...,g_b_0_t,1,0,3,2019-12-31 18:37:49,ADHD,3,3,interactive
1,1,ei6wbx,Excrutiating urgency to get out of the car.,Does anyone else feel an excrutiating urgency ...,amberaubade,1,0,7,2019-12-31 18:40:32,ADHD,6,6,interactive
2,2,ei6xa3,Happy new year my guys love you all x,That's the decade done! Bloody hell it's been ...,rstar345,1,0,0,2019-12-31 18:42:26,ADHD,0,0,isolated
3,3,ei7dle,How much has the mandatory drug tests usually ...,I finally decided to get back on meds (haven’t...,Bgeaz,1,0,5,2019-12-31 19:14:50,ADHD,5,3,interactive
4,4,ei7ee5,Recently Diagnosed- any advice?,I am currently in my 2nd year of undergrad and...,alyssaj99,1,0,2,2019-12-31 19:16:27,ADHD,2,2,non-interactive


In [ ]:
def sub_str(s):
  return s[3:]

In [ ]:
comment_df['parent_id'] =comment_df['parent_id'].apply(sub_str)

In [ ]:
parent_comments=dict(zip(comment_df.comment_id, comment_df.parent_id))

In [ ]:
# my_dict = {k:list(v) for k,v in zip(post_df['sub_id'], post_df.drop(columns='sub_id').values)}

In [ ]:
post_df.columns

Index(['Unnamed: 0', 'sub_id', 'title', 'body', 'author', 'score', 'awards',
       'numComms', 'created', 'subreddit', 'actual_com', 'no_of_posters',
       'post_category'],
      dtype='object')

In [ ]:
# for k,v in zip(post_df['sub_id'], post_df.drop(columns='sub_id').values):
#     print(k,v)
#     break

my_dict = {}
df = post_df.reset_index()  # make sure indexes pair with number of rows
for index, row in df.iterrows():
    my_dict[row['sub_id']] = {
        'title': row['title'], 
        'body': row['body'],
        'author': row['author'],
        'score': row['score'],
        'awards':row['awards'],
        'numComms': row['numComms'],
        'created': row['created'],
        'subredit': row['subreddit'],
        'actual_com': row['actual_com'],
        'number_of_posters': row['no_of_posters'],
        'post_category': row['post_category']
    }

In [ ]:
comment_df.head()

,post_id,body,author,is_op,created,score,awards,comment_id,parent_id
0,ei6uyt,I think that's actually a pretty common type o...,crisis_loitering,False,2020-01-01 05:26:15,1,0,fcpaoys,ei6uyt
1,ei6uyt,Thanks for this - I have been thinking about j...,g_b_0_t,True,2019-12-31 21:33:25,1,0,fcob8hk,fco58cc
2,ei6uyt,Try journaling or writing a diary. I like usi...,heywire84,False,2019-12-31 20:31:55,1,0,fco58cc,ei6uyt
3,ei6wbx,I'm glad you know you're not alone!,amberaubade,True,2020-01-10 00:13:17,1,0,fdoxlrn,fdl6xv3
4,ei6wbx,I really thought I was alone in this and that ...,BogusDou,False,2020-01-08 21:14:44,1,0,fdl6xv3,ei6wbx


In [ ]:
for id in all_post_ids:
  spec_com = comment_df[comment_df['post_id']==id]
  my_dict[id]['comment_ids'] = []
  my_dict[id]['comment_ids'].extend(list(spec_com['comment_id']))

In [ ]:
my_dict['eich0e']

{'actual_com': 4,
 'author': 'ihatelifejustkillme',
 'awards': 0,
 'body': "My body is refusing to rat any thing unless it's a really small bite what's wrong with me",
 'comment_ids': ['fcr26o3', 'fcoxr22', 'fcoxqev', 'fcoxmz8'],
 'created': '2020-01-01 02:05:24',
 'numComms': 4,
 'number_of_posters': 4,
 'post_category': 'interactive',
 'score': 1,
 'subredit': 'selfhelp',
 'title': "What's wrong with me"}

In [ ]:
comment_df.head()

,post_id,body,author,is_op,created,score,awards,comment_id,parent_id
0,ei6uyt,I think that's actually a pretty common type o...,crisis_loitering,False,2020-01-01 05:26:15,1,0,fcpaoys,ei6uyt
1,ei6uyt,Thanks for this - I have been thinking about j...,g_b_0_t,True,2019-12-31 21:33:25,1,0,fcob8hk,fco58cc
2,ei6uyt,Try journaling or writing a diary. I like usi...,heywire84,False,2019-12-31 20:31:55,1,0,fco58cc,ei6uyt
3,ei6wbx,I'm glad you know you're not alone!,amberaubade,True,2020-01-10 00:13:17,1,0,fdoxlrn,fdl6xv3
4,ei6wbx,I really thought I was alone in this and that ...,BogusDou,False,2020-01-08 21:14:44,1,0,fdl6xv3,ei6wbx


In [ ]:
comments=dict(zip(comment_df.comment_id, comment_df.body))

In [ ]:
posts=dict(zip(post_df.sub_id, post_df.body))

In [ ]:
ids_and_body ={}
ids_and_body.update(comments)
ids_and_body.update(posts)

In [ ]:
ids_and_body['eich0e']

"My body is refusing to rat any thing unless it's a really small bite what's wrong with me"

In [ ]:
interactive_post_ids = list(interactive_post_ids)
isolated_post_ids = list(isolated_post_ids)
non_interactive_ids = list(non_interactive_ids)

In [ ]:
comments_by_op = set(comment_df[comment_df['is_op']==True]['comment_id'])

In [ ]:
len(comments_by_op)

12191

# extract posts and their primary comments

In [ ]:
for id in interactive_post_ids:
  post_info = my_dict[id]
  post_body = my_dict[id]['body']
  post_title = my_dict[id]['title']
  comment_list = my_dict[id]['comment_ids']
  primary_comments=[]
  for c in comments:
    if(parent_comments[c]==id):
      primary_comments.append(ids_and_body[c])

op responses and comments for interactive posts

In [ ]:
count=0;
for id in (interactive_post_ids):
  post_info = my_dict[id]
  post_body = post_info[1]
  comment_list = post_info[-1]
  op_comment_reply_pairs=[]
  for c in comment_list:
    if(parent_comments[c] in (comments_by_op)):
      op_comment_reply_pairs.append([ids_and_body[parent_comments[c]],ids_and_body[c]])
  count+=len(op_comment_reply_pairs)
print("Total op comment reply pairs:", count)

Total op comment reply pairs: 6564
